In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import pickle
import seaborn as sns

import holidays


In [ ]:
# GET INDEXES OF DATA SUBSEQUENCES (ONE SUBSEQUENCE INCLUDES LAGGED (X) AND HORIZON (Y) LOADS)

def get_subsequence_indexes(in_data_df, window_size, step_size):
    # in_data_df = full (or train/test) dataset, window_size = number of lagged values + predicted (horizon) values, step_size = spacing between datapoints

    last_index = len(in_data_df)
    
    subseq_first_idx = 0 # Subsequence start and end index
    subseq_last_idx = window_size
    
    subseq_indexes = []
    
    while subseq_last_idx <= last_index: # Divide all data into subsequences (and get their indexes)

        subseq_indexes.append((subseq_first_idx, subseq_last_idx))
        
        subseq_first_idx += step_size
        subseq_last_idx += step_size

    return subseq_indexes

In [ ]:
# GET X,Y DATA SPLIT (EVERY DATAPOINT IS MADE UP OF SUB-SEQUENCE)

def get_xy_lagged(subseq_indexes, load_data, horizon_size, lag_size):

    for i, idx in enumerate(subseq_indexes):

        # Create subsequences
        subsequence = load_data[idx[0]:idx[1]] # Flat np array

        xi = subsequence[0: lag_size]
        yi = subsequence[lag_size: lag_size + horizon_size]

        if i == 0: # No existing array to append to
            y = np.array([yi]) # Turn y and x into rows, to make an array of arrays
            x = np.array([xi])

        else:
            y = np.concatenate((y, np.array([yi])), axis=0) # shape (datapoints, horizon)
            x = np.concatenate((x, np.array([xi])), axis=0) # shape (datapoints, input features)

    return x, y

In [ ]:
# GET DATETIME FEATURES

def create_dt_features(df):
    df_c = df.copy()
    df_c['Hour'] = df_c.index.hour
    df_c['Workday'] = df_c.index.map(lambda x: 0 if (x in holidays.Netherlands() or x.dayofweek in (5,6)) else 1) # 1 if workday, 0 if holiday or weekend
    df_c['Dayofweek'] = df_c.index.dayofweek
    df_c['Quarter'] = df_c.index.quarter
    df_c['Month'] = df_c.index.month
    df_c['Dayofyear'] = df_c.index.dayofyear
    df_c['Dayofmonth'] = df_c.index.day
    return df_c

In [ ]:
# GET CYCLICAL ENCODING

def cyclical_encoding(df, features):
    df_c = df.copy()
    
    for f in features:
        total_values = df_c[f].max() # E.g. total months = 12, starting at 1

        if df_c[f].min() == 0: # If first value is 0, total values is 1 more e.g. 24 hours
            total_values += 1 

        df_c[f + '_cos'] = np.cos(2*np.pi* df_c[f]/ total_values) # Encode into cos and sin values, that way end value is close to start value
        df_c[f + '_sin'] = np.sin(2*np.pi* df_c[f]/ total_values)
    
    return df_c

In [ ]:
# GET FORMATTED DATA (IN DATE-TIME) AND TRAIN-TEST SPLIT

def date_formatting(bus):

    df = pd.read_csv(f"./data/bus_{bus}_load.csv")
    df.index = pd.to_datetime(df.index, unit='h', origin=pd.Timestamp("2018-01-01"))
    df.index.name = "Time"
    df = create_dt_features(df)
    df = cyclical_encoding(df, features=["Hour", "Dayofweek", "Quarter", "Month", "Dayofyear", "Dayofmonth"])

    # training_data = df[df.index < split_date] # Splitting data now will cause it to lose 24 training datapoints and 24*7 test datapoints
    # test_data = df[df.index >= split_date]

    return df

def allbus_date_formatting(bus_init=1, bus_final=28):
    
    allbus_df = {}

    for b in range(bus_init, bus_final + 1):
        allbus_df[b] = date_formatting(b)

    return allbus_df
    

In [ ]:
# GET X,Y DATA 

def get_xy(in_data_df, step_size, horizon_size, hyperparameters):

    subseq_indexes = get_subsequence_indexes(in_data_df = in_data_df, window_size = hyperparameters["lag_size"] + horizon_size, step_size = step_size)
    
    lagged_x, y = get_xy_lagged(subseq_indexes=subseq_indexes, load_data=in_data_df[hyperparameters["selected_features"][0]].to_numpy(), 
                                horizon_size=horizon_size, lag_size=hyperparameters["lag_size"])     
    
    no_lag_features = in_data_df[hyperparameters["selected_features"][1:]].to_numpy() # Array of features that are not lagged, by rows (each row a timestep)

    first_timestep = hyperparameters["lag_size"] # Datapoints start after all lagged values can be obtained
    last_timestep = len(in_data_df) - (horizon_size - 1) # Last datapoint until it is possible to obtain all horizon values (horizon_size - 1)

    x = np.append(lagged_x, no_lag_features[first_timestep: last_timestep], axis=1) # Append no-lag features to lagged load features

    return x, y


def get_allbus_xy(allbus_in_data, hyperparameters, bus_init=1, bus_final=28, step_size=1, horizon_size=24):

    allbus_x, allbus_y = {}, {}

    for b in range(bus_init, bus_final + 1):
        allbus_x[b], allbus_y[b] = get_xy(allbus_in_data[b], step_size, horizon_size, hyperparameters)
    
    return allbus_x, allbus_y


In [ ]:
# GET X,Y SPLIT IN TRAIN, TEST

def split_train_test(x, y, lag_size, split_date="2018-10-16", train_val_split=0.8): # Timestep 6912

    original_timestep = (pd.Timestamp(split_date) - pd.Timestamp("2018-01-01 00:00:00")).total_seconds()/3600 # Get original timestep (hour) from split date
    split_timestep = int(original_timestep - lag_size) # Split timestep in the new dataframe (starts at timestep lag_size)
    
    x_tr = x[:split_timestep]
    y_tr = y[:split_timestep]

    x_train, x_val = x_tr[:int(train_val_split * len(x_tr))], x_tr[int(train_val_split * len(x_tr)):] # Split in train and validation
    y_train, y_val = y_tr[:int(train_val_split * len(y_tr))], y_tr[int(train_val_split * len(y_tr)):]

    x_test = x[split_timestep:]
    y_test = y[split_timestep:]

    return x_train, y_train, x_val, y_val, x_test, y_test


def allbus_split_train_test(allbus_x, allbus_y, lag_size, bus_init=1, bus_final=28, split_date="2018-10-16"):

    allbus_x_train, allbus_y_train, allbus_x_val, allbus_y_val, allbus_x_test, allbus_y_test = {}, {}, {}, {}, {}, {}

    for b in range(bus_init, bus_final + 1):
        allbus_x_train[b], allbus_y_train[b], allbus_x_val[b], allbus_y_val[b], allbus_x_test[b], allbus_y_test[b] = \
            split_train_test(allbus_x[b], allbus_y[b], lag_size, split_date)
    
    return allbus_x_train, allbus_y_train, allbus_x_val, allbus_y_val, allbus_x_test, allbus_y_test


In [ ]:
# GET AND STORE ALL MODELS FOR ALL BUSSES AND THEIR TRAIN/TEST SCORES

def get_model(x_train, y_train, x_val, y_val, x_test, y_test, hyperparameters, score_function):
     
    model = XGBRegressor(n_estimators=hyperparameters["n_estimators"], max_depth=hyperparameters["max_depth"], subsample=hyperparameters["subsample"], 
                            gamma=hyperparameters["gamma"], reg_lambda=hyperparameters["lambda"], objective="reg:squarederror", tree_method="hist", 
                            verbosity=3, learning_rate=hyperparameters["learning_rate"])

    trained_model = MultiOutputRegressor(model).fit(X=x_train, y=y_train, verbose=True) # Evaluate on validation data

    train_forecasts = trained_model.predict(x_train)
    train_score = score_function(y_train, train_forecasts)

    valid_forecasts = trained_model.predict(x_val)
    valid_score = score_function(y_val, valid_forecasts)

    test_forecasts = trained_model.predict(x_test)
    test_score = score_function(y_test, test_forecasts)

    model_score = {"Train score": train_score, "Validation score": valid_score, "Test score": test_score}

    return trained_model, model_score


def get_models(models_datapath, hyperparameters, score_function, bus_init=1, bus_final=28, horizon_size=24, step_size=1, 
               allbus_x_train=None, allbus_y_train=None, allbus_x_val=None, allbus_y_val=None, allbus_x_test=None, allbus_y_test=None):

    trained_models = {}
    model_scores = {}
    
    for bus in range(bus_init, bus_final + 1):

        if (allbus_x_train != None) and (allbus_y_train != None) and (allbus_x_val != None) and (allbus_y_val != None) and \
            (allbus_x_test != None) and (allbus_y_test != None): # If they're all defined
            x_train, y_train, x_val, y_val, x_test, y_test = allbus_x_train[bus], allbus_y_train[bus], allbus_x_val[bus], allbus_y_val[bus], \
                allbus_x_test[bus], allbus_y_test[bus]
        
        else:
            print("Missing input data to get_models()")

            df = date_formatting(bus)

            x, y = get_xy(df, step_size, horizon_size, hyperparameters)

            x_train, y_train, x_val, y_val, x_test, y_test = split_train_test(x, y, hyperparameters["lag_size"])
        
        
        trained_model, model_scores[bus] = get_model(x_train, y_train, x_val, y_val, x_test, y_test, hyperparameters, score_function)
        
        trained_models[bus] = trained_model

        # Store models
        with open(f"{models_datapath}/MOR_bus{bus}.pkl", "wb") as f1:
            pickle.dump(trained_model, f1)

    # Store model scores
    with open(f"{models_datapath}/MOR_scores.pkl", "wb") as f2:
                pickle.dump(model_scores, f2)

    return trained_models, model_scores


In [ ]:
# GET SINGLE-BUS PREDICTION, WITH TIME RELATIVE TO START OF TEST DATA

def predict_bus(x_test, y_test, bus, t_init, t_duration, models_datapath, score_function): # Starting at 2018-10-16 00:00:00 by default

    # if not (x_test.any() and y_test.any()): # If x_test or y_test are not provided, get it
    #     if not test_data.any(): # If test_data is not provided, get it
    #         _, _, test_data = date_formatting(bus=bus)

    #     x_test, y_test = get_xy(test_data, step_size, horizon_size, hyperparameters)

    x = x_test[t_init: t_init + t_duration]
    y = y_test[t_init: t_init + t_duration]
    
    with open(f"{models_datapath}/MOR_bus{bus}.pkl", "rb") as f1:
        model = pickle.load(f1)

    with open(f"{models_datapath}/MOR_scores.pkl", "rb") as f2:
        model_scores = pickle.load(f2)

    total_model_score = model_scores[bus]

    forecast = model.predict(x)
    prediction_score = score_function(y, forecast)

    return forecast, y, x, prediction_score, total_model_score

In [ ]:
# GET PREDICTIONS FOR ALL BUSSES, WITH TIME RELATIVE TO START OF TEST DATA

def predict_allbus(allbus_x_test, allbus_y_test, t_init, t_duration, models_datapath, out_path, score_function, bus_init=1, bus_final=28,
                   actual_bus_numbers=np.array([None])): # t_init w.r.t. test data start
    
    # if (allbus_x_test != None) and (allbus_y_test != None): # If allbus_x_test and allbus_y_test are provided, disregard allbus_test_data. Else provide it.
    #     allbus_test_data = None

    bus_predictions, bus_y_actual, bus_x_actual, bus_prediction_score, model_score = {}, {}, {}, {}, {}

    
    for bus in range(bus_init, bus_final + 1): # Get individual bus predictions
        bus_predictions[bus], bus_y_actual[bus], bus_x_actual[bus], bus_prediction_score[bus], model_score[bus] = predict_bus(allbus_x_test[bus], 
            allbus_y_test[bus], bus, t_init, t_duration, models_datapath, score_function)

    if not actual_bus_numbers.any(): # If actual bus numbers not defined
        actual_bus_numbers = np.array([i for i in range(bus_init, bus_final + 1)])
    
    predictions_df = pd.concat([pd.concat([pd.Series(bus_predictions[b][t]).to_frame().T for b in range(bus_init, bus_final + 1)], 
                                          axis=0).set_index(actual_bus_numbers) for t in range(t_init, t_init + t_duration)], axis=0) 
    
    # Rows of 24-hour predictions per bus concatenated into pd for every timestep, then pds concatenated for all timesteps

    # predictions.set_index(actual_bus_numbers)

    predictions_df.to_csv(out_path + "/forecast.csv")

    return predictions_df, bus_predictions, bus_y_actual, bus_x_actual, bus_prediction_score, model_score


In [ ]:
# DEFINE HYPERPARAMETERS AND GET ALL TRAINING AND TESTING X AND Y DATA

hyperparameters = {"lag_size": 7*24, "n_estimators": 100, "max_depth": 3, "subsample": 1,
                   "selected_features": ["Load", "Hour_cos", "Hour_sin", "Workday"], 
                   "learning_rate": 0.1, "gamma": 0, "lambda": 1,
                   "early_stopping_rounds": 10}

# n_estimators determines how many rounds of training (how many new trees can be made)
# early_stopping_rounds determines how many rounds of training can go before validation set score improves from its best score. Needs validation set 
#   to be included when training/fitting. If there is any number, the model with iteration that has best score on validation set will be used when 
#   predicting, even if training is not early stopped. In that case, it will be the same model as changing n_estimators to be the same as best validation iter

data_store_path = "./train_test_data"

# Get train-test split for all busses
allbus_df = allbus_date_formatting(bus_init=1, bus_final=28)

# Get x,y split for all busses
allbus_x, allbus_y = get_allbus_xy(allbus_df, hyperparameters, bus_init=1, bus_final=28, step_size=1, horizon_size=24) 
# Uses hyperparameters lag_size, selected_features

# Get x,y split for all busses test data
allbus_x_train, allbus_y_train, allbus_x_val, allbus_y_val, allbus_x_test, allbus_y_test = \
    allbus_split_train_test(allbus_x, allbus_y, hyperparameters["lag_size"], bus_init=1, bus_final=28)

# Store the data in a file
with open(f"{data_store_path}/allbus_train_test.pkl", "wb") as f:
    pickle.dump((allbus_x_train, allbus_y_train, allbus_x_val, allbus_y_val, allbus_x_test, allbus_y_test), f)

In [ ]:
# Load back data

data_store_path = "./train_test_data"

with open(f"{data_store_path}/allbus_train_test.pkl", "rb") as f2:
    (allbus_x_train, allbus_y_train, allbus_x_val, allbus_y_val, allbus_x_test, allbus_y_test) = pickle.load(f2)

In [ ]:
allbus_x[1].shape 
# Should be be [8760 - 24 + 1 (last 23 datapoints can't be used) - 24*7 (first 24*7 datapoints can't be used), 
# by 24*7=168 lag features + no_lag_features features] = [8569, 168 + no_lag_features]

In [ ]:
# TRAIN AND STORE THE MODELS FOR ALL BUSSES

trained_models, model_scores = get_models(models_datapath="./MOR_v1", hyperparameters=hyperparameters, score_function=mean_absolute_error, 
                                allbus_x_train=allbus_x_train, allbus_y_train=allbus_y_train, allbus_x_val=allbus_x_val,
                                 allbus_y_val=allbus_y_val, allbus_x_test=allbus_x_test, allbus_y_test=allbus_y_test)

In [ ]:
# GET BENCHMARK SCORES

with open("./benchmark_scores.pkl", "rb") as f2:
    allbus_benchmarks = pickle.load(f2)

In [ ]:
# COMPARE BENCHMARK AND ACTUAL SCORES

score_compare = {}
approved_trains = [1 for _ in range(28)]
approved_tests = [1 for _ in range(28)]

for b in range(1, 29):

    train_predict = trained_models[b].predict(allbus_x_train[b])
    test_predict = trained_models[b].predict(allbus_x_test[b])

    train_score = mean_absolute_error(train_predict, allbus_y_train[b])
    test_score = mean_absolute_error(test_predict, allbus_y_test[b])

    score_compare[b] = {"Train": train_score, "Test": test_score}

    for t in [1, 24, 24*7]:
        if allbus_benchmarks[b][f"{t}h"]["Train"] <= train_score:
            approved_trains[b-1] = 0
    
        if allbus_benchmarks[b][f"{t}h"]["Test"] <= test_score:
            approved_tests[b-1] = 0


In [ ]:
# Testing getting a single bus prediction

hyperparameters = {"lag_size": 7*24, "n_estimators": 20, "max_depth": 6, "subsample": 0.5, "min_child_weight": 1, "selected_features": ["Load", "Hour"], 
                   "learning_rate": 0.2, "early_stopping_rounds": 10}

y_pred1, y_actual1, x_actual1, mae1, model_score1 = predict_bus(allbus_x_test[1], allbus_y_test[1], bus=1, t_init=0, t_duration=24, models_datapath="./MOR_v1", 
                                                               score_function=mean_absolute_error)
print(f"Predictions: {y_pred1[0]}")
print(f"Actual: {y_actual1[0]}")
print(f"Prediction MAE: {mae1}")
print(f"Model MAE: {model_score1}")

In [ ]:
actual_bus_numbers = np.array([2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34])
actual_bus_numbers += 1

P_df, P, Y, X, P_S, M_S  = predict_allbus(t_init=0, t_duration=48, models_datapath="./MOR_v1", out_path="./simulation_input", 
                                          hyperparameters=hyperparameters, score_function=mean_absolute_error, allbus_x_test=allbus_x_test, 
                                          allbus_y_test=allbus_y_test, actual_bus_numbers=actual_bus_numbers)

In [ ]:
P_df.head()

In [ ]:
print(P_S)
print(M_S)

In [ ]:
def plot(time, forecast: np.array, actual: np.array):
    fontsize = 16
    plot_df = pd.DataFrame({"Forecast" : forecast, "Real value" : actual}, index=range(time))

    fig = plt.figure(figsize=(20,12))
    plt.plot(plot_df.index, plot_df["Forecast"], label="Forecast")
    plt.plot(plot_df.index, plot_df["Real value"], label="Real value")

    plt.xlabel('Time [h]', fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.ylabel("Load [MW]", fontsize=fontsize)
    plt.grid(True)
    plt.legend(fontsize=fontsize)
    plt.tight_layout()

In [ ]:
fontsize = 16
timestep = 24*10
plot_df = pd.DataFrame({"Forecast 0h" : trained_models[2].predict(allbus_x_test[2])[timestep], "Real value" : allbus_y_test[2][48]}, index=range(24))



fig = plt.figure(figsize=(20,12))
plt.plot(plot_df.index, plot_df["Real value"], label="Real value")
plt.plot(plot_df.index, plot_df["Forecast 0h"], label="Forecast 0h")

x = [i for i in range(24)]
for time in [4, 8, 12, 16, 20]:
    plot_kh = trained_models[2].predict(allbus_x_test[2])[timestep+time][:-time]
    plt.plot(x[time:], plot_kh, label=f"Forecast {time}h")




plt.xlabel('Time [h]', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.ylabel("Load [MW]", fontsize=fontsize)
plt.grid(True)
plt.legend(fontsize=fontsize)
plt.tight_layout()